In [1]:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import re
import numpy as np

In [2]:
url = 'https://codeup.com/codeups-data-science-career-accelerator-is-here/'
headers = {'User-Agent': 'Codeup Data Science'} 
response = get(url, headers=headers)

In [3]:
print(response.text[:300])

<!DOCTYPE html><html lang="en-US"><head ><meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1" /> <script>function fvmuag(){if(navigator.userAgent.match(/x11.*fox\/54|oid\s4.*xus.*ome\/62|oobot|ighth|tmetr|eadles|ingdo/i))return!1;if(navigator.userAgent.match(/x


In [4]:
print(response.content[:300])

b'<!DOCTYPE html><html lang="en-US"><head ><meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1" /> <script>function fvmuag(){if(navigator.userAgent.match(/x11.*fox\\/54|oid\\s4.*xus.*ome\\/62|oobot|ighth|tmetr|eadles|ingdo/i))return!1;if(navigator.userAgent.match(/x'


In [5]:
# Make a soup variable holding the response content
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
#soup.title.string gets the page's title (the same text in the browser tab for a page, this is the <title> element
soup.title.string

'Codeup’s Data Science Career Accelerator is Here! - Codeup'

In [7]:
# soup.prettify() is useful to print in case you want to see the HTML
soup.prettify()[:500]

'<!DOCTYPE html>\n<html lang="en-US">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width, initial-scale=1" name="viewport"/>\n  <script>\n   function fvmuag(){if(navigator.userAgent.match(/x11.*fox\\/54|oid\\s4.*xus.*ome\\/62|oobot|ighth|tmetr|eadles|ingdo/i))return!1;if(navigator.userAgent.match(/x11.*ome\\/75\\.0\\.3770\\.100/i)){var e=screen.width,t=screen.height;if("number"==typeof e&&"number"==typeof t&&862==t&&1367==e)return!1}return!0}\n  </script>\n  <style id="aoatfcss" media="all'

In [8]:
# soup.find_all("a") find all the anchor tags, or whatever argument is specified.
soup.find_all("a")

[<a class="jupiterx-a11y jupiterx-a11y-skip-navigation-link" href="#jupiterx-primary">Skip to content</a>,
 <a class="raven-site-logo-link" href="https://codeup.com"> <img alt="Codeup" class="raven-site-logo-desktop raven-site-logo-tablet raven-site-logo-mobile lazyload" data-no-lazy="1" data-sizes="(max-width: 226px) 100vw, 226px" data-src="https://750092.smushcdn.com/1449913/wp-content/uploads/2018/08/logo.png?lossy=1&amp;strip=1&amp;webp=1" data-srcset="https://750092.smushcdn.com/1449913/wp-content/uploads/2018/08/logo.png?lossy=1&amp;strip=1&amp;webp=1 226w, " src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=="/><noscript><img alt="Codeup" class="raven-site-logo-desktop raven-site-logo-tablet raven-site-logo-mobile" data-no-lazy="1" sizes="(max-width: 226px) 100vw, 226px" src="https://750092.smushcdn.com/1449913/wp-content/uploads/2018/08/logo.png?lossy=1&amp;strip=1&amp;webp=1" srcset="https://750092.smushcdn.com/1449913/wp-content/uploads/2018/08/log

In [9]:
# soup.find("h1") finds the first matching element
soup.find("h1")

<h1 class="jupiterx-post-title" itemprop="headline">Codeup’s Data Science Career Accelerator is Here!</h1>

In [10]:
# soup.get_text() gets the text from within a matching piece of soup/HTML
soup.get_text()

' function fvmuag(){if(navigator.userAgent.match(/x11.*fox\\/54|oid\\s4.*xus.*ome\\/62|oobot|ighth|tmetr|eadles|ingdo/i))return!1;if(navigator.userAgent.match(/x11.*ome\\/75\\.0\\.3770\\.100/i)){var e=screen.width,t=screen.height;if("number"==typeof e&&"number"==typeof t&&862==t&&1367==e)return!1}return!0} <style>div{box-sizing:border-box}div{box-sizing:border-box}@font-face{font-family:FontAwesome;src:url(//codeup.com/wp-content/plugins/elementor/assets/lib/font-awesome/css/../fonts/fontawesome-webfont.eot?#iefix&v=4.7.0) format(\'embedded-opentype\'),url(//codeup.com/wp-content/plugins/elementor/assets/lib/font-awesome/css/../fonts/fontawesome-webfont.woff2?v=4.7.0) format(\'woff2\'),url(//codeup.com/wp-content/plugins/elementor/assets/lib/font-awesome/css/../fonts/fontawesome-webfont.woff?v=4.7.0) format(\'woff\'),url(//codeup.com/wp-content/plugins/elementor/assets/lib/font-awesome/css/../fonts/fontawesome-webfont.ttf?v=4.7.0) format(\'truetype\'),url(//codeup.com/wp-content/plugin

In [15]:
urls = ['https://codeup.com/codeups-data-science-career-accelerator-is-here/',
        'https://codeup.com/data-science-myths/',
        'https://codeup.com/data-science-vs-data-analytics-whats-the-difference/',
        'https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/',
        'https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/']

def get_blog_articles(urls, cache=False):
    '''
    This function takes in a list of Codeup Blog urls and a parameter
    with default cache == False which returns a df from a csv file.
    If cache == True, the function scrapes the title and text for each url, 
    creates a list of dictionaries with the title and text for each blog, 
    converts list to df, and returns df.
    '''
    if cache == False:
        df = pd.read_csv('big_blogs.csv', index_col=0)
    else:
        headers = {'User-Agent': 'Codeup Bayes Data Science'} 

        # Create an empty list to hold dictionaries
        articles = []

        # Loop through each url in our list of urls
        for url in urls:

            # get request to each url saved in response
            response = get(url, headers=headers)

            # Create soup object from response text and parse
            soup = BeautifulSoup(response.text, 'html.parser')

            # Save the title of each blog in variable title
            title = soup.find('h1', itemprop='headline').text

            # Save the text in each blog to variable text
            text = soup.find('div', itemprop='text').text

            # Create a dictionary holding the title and text for each blog
            article = {'title': title, 'content': text}

            # Add each dictionary to the articles list of dictionaries
            articles.append(article)
            
        # convert our list of dictionaries to a df
        df = pd.DataFrame(articles)

        # Write df to csv file for faster access
        df.to_csv('big_blogs.csv')
    
    return df

In [16]:
get_blog_articles(urls, cache=False)

,title,content
0,Codeup’s Data Science Career Accelerator is Here!,The rumors are true! The time has arrived. Cod...
1,Data Science Myths,By Dimitri Antoniou and Maggie GiustData Scien...
2,Data Science VS Data Analytics: What’s The Dif...,"By Dimitri AntoniouA week ago, Codeup launched..."
3,10 Tips to Crush It at the SA Tech Job Fair,10 Tips to Crush It at the SA Tech Job FairSA ...
4,Competitor Bootcamps Are Closing. Is the Model...,Competitor Bootcamps Are Closing. Is the Model...


## Inshorts News Articles
Goal: Write a function that scrapes the news articles for the following topics:

 - Business
 - Sports
 - Technology
 - Entertainment

In [20]:
url = 'https://inshorts.com/en/read/entertainment'

response = get(url)
response.ok

True

In [21]:
soup = BeautifulSoup(response.text, 'html.parser')

In [22]:
# Scrape a ResultSet of all the news cards on the page and look at first card

cards = soup.find_all('div', class_='news-card')
print(type(cards))
cards[0]

<class 'bs4.element.ResultSet'>


<div class="news-card z-depth-1" itemscope="" itemtype="http://schema.org/NewsArticle">
<span content="" itemid="https://inshorts.com/en/news/fir-against-sa-re-ga-ma-pa-runnerup-from-bdesh-for-remarks-against-pm-modi-1590658280071" itemprop="mainEntityOfPage" itemscope="" itemtype="https://schema.org/WebPage"></span>
<span itemprop="author" itemscope="itemscope" itemtype="https://schema.org/Person">
<span content="Apaar Sharma" itemprop="name"></span>
</span>
<span content="FIR against Sa Re Ga Ma Pa runner-up from B’desh for remarks against PM Modi" itemprop="description"></span>
<span itemprop="image" itemscope="" itemtype="https://schema.org/ImageObject">
<meta content="https://static.inshorts.com/inshorts/images/v1/variants/jpg/m/2020/05_may/28_thu/img_1590656019484_320.jpg?" itemprop="url"/>
<meta content="864" itemprop="width"/>
<meta content="483" itemprop="height"/>
</span>
<span itemprop="publisher" itemscope="itemscope" itemtype="https://schema.org/Organization">
<span conten

In [23]:
# Save the title of each news card to list titles

titles = []
for card in cards:
    title = card.find('span', itemprop='headline').text
    titles.append(title)
    
titles[:5]

['FIR against Sa Re Ga Ma Pa runner-up from B’desh for remarks against PM Modi',
 'Ravi Mohan Saini, who won ₹1 cr in KBC Junior at 14, becomes SP of Porbandar',
 "Preksha was upset over lockdown extension, she didn't like sitting idle: Father",
 "Didn't wear this to 'hide my curves': Maanvi on her pic used for clothing store's ad",
 'I promise to make our fellow Punjabis proud: Sonu Sood as Punjab CM praises him']

In [24]:
# Save the author of the news card to list authors

authors = []
for card in cards:
    author = card.find('span', class_='author').text
    authors.append(author)
    
authors[:5]

['Apaar Sharma', 'Pragya Swastik', 'Daisy Mowke', 'Daisy Mowke', 'Udit Gupta']

In [25]:
# Save the text of each article to a list of texts

texts = []
for card in cards:
    text = card.find('div', itemprop='articleBody').text
    texts.append(text)
    
texts[:2]

["Tripura Police has registered an FIR against Bangladeshi singer Mainul Ahsan Noble, aka Noble, for allegedly making derogatory remarks against Prime Minister Narendra Modi on social media. Noble had finished as second runner-up on Zee Bangla's music reality show Sa Re Ga Ma Pa last year. An engineering student in Gujarat said he filed the complaint against Noble on Monday.",
 "IPS officer Ravi Mohan Saini, who won ₹1 crore in KBC Junior when he was 14 years old, took charge as Superintendent of Police, Porbandar, Gujarat on Tuesday. Saini, who is now 33 years old, qualified for Indian Police Service in 2014 with AIR 461. A native of Rajasthan's Alwar, Saini is the son of a retired Navy officer."]

In [26]:
# Create an empty list, articles, to hold the dictionaries for each article
articles = []

# Loop through each news card on the page and get what we want
for card in cards:
    title = card.find('span', itemprop='headline' ).text
    author = card.find('span', class_='author').text
    content = card.find('div', itemprop='articleBody').text
    
    # Create a dictionary, article, for each news card
    article = {'title': title, 'author': author, 'content': content}
    
    # Add the dictionary, article, to our list of dictionaries, articles.
    articles.append(article)

In [27]:
def get_news_articles(cache=False):
    '''
    This function uses a cache parameter with default cache == False to give the option of 
    returning in a df of inshorts topics and info by reading a csv file or
    of doing a fresh scrape of inshort pages with topics business, sports, technology,
    and entertainment and writing the returned df to a csv file.
    '''
    # default to read in a csv instead of scrape for df
    if cache == False:
        df = pd.read_csv('articles.csv', index_col=0)
        
    # cache == True completes a fresh scrape for df    
    else:
    
        # Set base_url and headers that will be used in get request

        base_url = 'https://inshorts.com/en/read/'
        headers = {'User-Agent': 'Codeup Data Science'}
        
        # List of topics to scrape
        topics = ['business', 'sports', 'technology', 'entertainment']

        # Create an empty list, articles, to hold our dictionaries
        articles = []

        for topic in topics:

            # Get a response object from the main inshorts page
            response = get(base_url + topic, headers=headers)

            # Create soup object using response from inshort
            soup = BeautifulSoup(response.text, 'html.parser')

            # Scrape a ResultSet of all the news cards on the page
            cards = soup.find_all('div', class_='news-card')

            # Loop through each news card on the page and get what we want
            for card in cards:
                title = card.find('span', itemprop='headline' ).text
                author = card.find('span', class_='author').text
                content = card.find('div', itemprop='articleBody').text

                # Create a dictionary, article, for each news card
                article = ({'topic': topic, 
                            'title': title, 
                            'author': author, 
                            'content': content})

                # Add the dictionary, article, to our list of dictionaries, articles.
                articles.append(article)
            
        # Why not return it as a DataFrame?!
        df = pd.DataFrame(articles)
        
        # Write df to csv for future use
        df.to_csv('articles.csv')
    
    return df

In [29]:
get_news_articles(cache=True)

,topic,title,author,content
0,business,US firm buys Serum Institute parent's Czech un...,Krishna Veera Vanamali,US biotech firm Novavax has announced it's buy...
1,business,Microsoft in talks to buy 2.5% stake in Jio fo...,Anushka Dixit,Microsoft is in talks with Mukesh Ambani-led R...
2,business,Google in talks to buy 5% stake in Vodafone Id...,Krishna Veera Vanamali,Google is exploring an investment in Vodafone ...
3,business,Kent's Atta maker ad says 'maid's hands may be...,Pragya Swastik,Kent RO Systems withdrew an advertisement for ...
4,business,Abu Dhabi state fund in talks to invest $1 bil...,Anushka Dixit,Abu Dhabi state fund Mubadala Investment Compa...
...,...,...,...,...
95,entertainment,John is almost a superhero like Hulk in Satyam...,Udit Gupta,Director Milap Zaveri has said John Abraham is...
96,entertainment,Henry Cavill in talks to return as Superman in...,Udit Gupta,British actor Henry Cavill is reportedly in ta...
97,entertainment,"Akshay sends money to 1,500 daily wagers' acco...",Atul Mishra,Akshay Kumar has donated ₹45 lakh to Cine & TV...
98,entertainment,Apple secures deal for Scorsese's next with Di...,Atul Mishra,Apple has secured a deal for Hollywood directo...


In [30]:
# Test our function to read in the df from `articles.csv`

df = get_news_articles(cache=False)
df.head()

,topic,title,author,content
0,business,US firm buys Serum Institute parent's Czech un...,Krishna Veera Vanamali,US biotech firm Novavax has announced it's buy...
1,business,Microsoft in talks to buy 2.5% stake in Jio fo...,Anushka Dixit,Microsoft is in talks with Mukesh Ambani-led R...
2,business,Google in talks to buy 5% stake in Vodafone Id...,Krishna Veera Vanamali,Google is exploring an investment in Vodafone ...
3,business,Kent's Atta maker ad says 'maid's hands may be...,Pragya Swastik,Kent RO Systems withdrew an advertisement for ...
4,business,Abu Dhabi state fund in talks to invest $1 bil...,Anushka Dixit,Abu Dhabi state fund Mubadala Investment Compa...
